In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, clear_output
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('DEEPSEEK_API_KEY')
base_url=os.getenv('DEEPSEEK_BASE_URL')
MODEL = "deepseek-chat"

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]
 
# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; Looks like you are using DeepSeek (R1) model.")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")
        
openai = OpenAI(api_key=api_key, base_url=base_url)

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
       
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]
    
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website),
        stream=True
    )
    print("Streaming response:")
    accumulated_content = ""  # Accumulate the content here
    for chunk in response:
        if chunk.choices[0].delta.content:  # Check if there's content in the chunk
            accumulated_content += chunk.choices[0].delta.content  # Append the chunk to the accumulated content
            clear_output(wait=True)  # Clear the previous output
            display(Markdown(accumulated_content))  # Display the updated content

def display_summary():
    url = str(input("Enter the URL of the website you want to summarize: "))
    summarize(url)

display_summary()